In [11]:
import os
import jieba
import re
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer, StopWordsRemover
from pyspark.ml.clustering import LDA
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import pandas as pd
import numpy as np

# 初始化 SparkSession
spark = SparkSession.builder \
    .appName("PySpark LDA Analysis") \
    .getOrCreate()

# 自定義分詞函數並去除標點符號和非中文字符
def jieba_tokenize(text):
    # 去除標點符號和非中文字符
    text = re.sub(r"[^\u4e00-\u9fa5]", "", text)  # 只保留中文字符
    return list(jieba.cut(text))

# 讀取 CSV 文件
csv_path = "/home/rrrrryeedie/final_sorted_output.csv"
data = spark.read.csv(csv_path, header=True, inferSchema=True)

# 篩選需要的欄位，例如 'title' 和 'body'
data = data.select('title', 'body').dropna()

# 將分詞函數轉換為UDF
jieba_udf = udf(jieba_tokenize, ArrayType(StringType()))
data = data.withColumn("words", jieba_udf(data["body"]))

# 停用詞處理（自定義更多停用詞）
stopwords = set([
    '的', '了', '在', '是', '和', '也', '有', '我', '他', '她', '它', '就', '都', '很', '而', '但', '不', '這', '那',
    '與', '為', '個', '被', '與', '對', '和', '或', '能', '要', '更', '來', '看', '和', '而且', '是', '你', '我們', '她們',
    '會', '查核', '說', '表示', '於', '打', '影片', '網傳', '月', '年',  # 已加入之前的高頻詞
    '傳言', '年月日', '照片', '指出', '可能', '等', '可以', '吃', '因此', '所', '日', '製', '後', '目前', '報導',
    '到', '使用', '畫面', '事件', '發布', '人', '中心', '報告', '發生', '造成', '社群', '平台', '規定', '內容', '文章', '圖', '傳', '事實', '是否',
    '之', '以上', '流傳', '上', '及', '錯誤', '從', '若', '檢視', '更新版', '近期', '該', '一', '二','圖片', '網站' , '如果', '不會','可','讓','隨時','更新',
    '找到','出現', '原始', '背景','誤訊息', '沒', '沒有', '多', '影響','以','長','錯','再','用','中','不是','含有','隻','日期','報告將','因為', 
    '較', '部分', '相關', '並非', '歲', '反應', '已','宣稱','處理', '流傳訊息', '反','均','建議','者','時','資料','效果','快','顯示','不要','拍','元',
    '不同','未','至', '指稱','進行','由','稱','文字','比','搭配','無法','需要','增加','訊息','一網','通過', '含量', '擷圖', '字', '事故','喝',
    '軍','研究', '高','根據'
])


def remove_stopwords(words):
    return [word for word in words if word not in stopwords]

# 去除停用詞
remove_stopwords_udf = udf(remove_stopwords, ArrayType(StringType()))
data = data.withColumn("filtered_words", remove_stopwords_udf(data["words"]))

# 設置 CountVectorizer
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="features", vocabSize=5000, minDF=5)  # 最小頻次為5
vectorized_model = vectorizer.fit(data)
vectorized_data = vectorized_model.transform(data)

# 設置 LDA 模型
lda = LDA(k=5, maxIter=10, featuresCol="features")  # k 是主題數量

# 訓練 LDA 模型
lda_model = lda.fit(vectorized_data)

# 取得主題描述
topics = lda_model.describeTopics(10) # 關鍵詞數量
topics_pd = topics.toPandas()

# 提取關鍵詞對應的字詞
vocab_list = vectorized_model.vocabulary
topics_pd["terms"] = topics_pd["termIndices"].apply(lambda indices: [vocab_list[idx] for idx in indices])

# 顯示每個主題的關鍵詞
for index, row in topics_pd.iterrows():
    print(f"主題 {index + 1}:")
    print("關鍵詞:", row["terms"])
    print("權重:", row["termWeights"])
    print("-" * 50)

# 使用 numpy 計算每個主題的關鍵詞數量總和
topic_keywords_count = topics_pd["termWeights"].apply(lambda x: np.sum(x)).values

# 儲存主題分佈結果到 CSV
topics_pd["total_keyword_weight"] = topic_keywords_count
topics_pd.to_csv("/home/rrrrryeedie/lda_topics.csv", index=False)

# 取得每篇文檔的主題分佈
doc_topics = lda_model.transform(vectorized_data)
doc_topics_pd = doc_topics.select("title", "topicDistribution").toPandas()

# 使用 numpy 處理每篇文檔的主題分佈矩陣
doc_topic_matrix = np.array(doc_topics_pd["topicDistribution"].tolist())
dominant_topics = np.argmax(doc_topic_matrix, axis=1)

# 添加每篇文檔的主要主題到 DataFrame
doc_topics_pd["dominant_topic"] = dominant_topics

# 儲存文檔主題分佈結果到 CSV
doc_topics_pd.to_csv("/home/rrrrryeedie/doc_topics.csv", index=False)

print("LDA 分析完成，結果已保存至 /home/rrrrryeedie/")


主題 1:
關鍵詞: ['進口', '公文', '台灣', '蛋', '中國', '公司', '選手', '關節', '雞', '澳洲']
權重: [0.008206884779654243, 0.005994135934372179, 0.005690675884453474, 0.005509691157327509, 0.004557340223090088, 0.004157882983824674, 0.00413030388545566, 0.003923864347230134, 0.0035436339540403715, 0.0031849361718670507]
--------------------------------------------------
主題 2:
關鍵詞: ['投票', '民眾', '台灣', '選舉', '專家', '寄生', '開票', '高雄市', '投開票', '安全']
權重: [0.012118184557641094, 0.006650489219584877, 0.005049848471347734, 0.0035408530426518626, 0.0035395679246442453, 0.0035326592734274812, 0.0033703285441612236, 0.0032981712481881665, 0.002982460841225665, 0.00286187428860488]
--------------------------------------------------
主題 3:
關鍵詞: ['病毒', '台灣', '地震', '疫情', '日本', '新冠', '感染', '口罩', '疫苗', '中國']
權重: [0.011550388201313865, 0.009867791577105942, 0.009215027040650598, 0.008155009923120798, 0.007763957908781, 0.006973153484242991, 0.005760020467209449, 0.005472496662501686, 0.003831112534118498, 0.003704749249668912]
-----

LDA 分析完成，結果已保存至 /home/rrrrryeedie/
